# Imports and config

In [1]:
import os
import pickle
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error

import config as cfg


warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

# Data preparation

### Load data

In [2]:
sales = pd.read_csv(cfg.FILENAMES['TRAIN_SALES'])

items = pd.read_csv(cfg.FILENAMES["ITEMS"])

categories = pd.read_csv(cfg.FILENAMES["ITEM_CATEGORIES"])

shops = pd.read_csv(cfg.FILENAMES["SHOPS"])

### Categories features